In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'smart-meters-in-london:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4021%2F3684057%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240314%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240314T163712Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2f5ac54159b9b9392f3c8a776491c987ee5e79c15ddfaa4276576c4853107088dd42e81b1dae33eeda08fcb3b0632fb0de97b075e68ccb9f55bdf242b0df37909b7c91ab4488607324f4d85dbaf9d344fb52d228b062e27a22c41712260b5c3e9f5f6cca5a3779a3716d2782bd2581fd46f18b952992e926edee34ea6c703cf0105b42c0b5bb4db42e7802ec7a008c23dd42496ff315cfadc8bf16ddc3838e586fb0bf6d8b015c4ccc56626caf4ad7acd219d4b81efcee5e2b0d4bc2cc4d5684670bc6cbf84af77187e27ecd74a622fa35186fee4566361c35835bc0209656429f3d31ac3284109f64ac946824ff22b253a8cc24e3f5b6955917bedbef22216d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Short-term residential load forecasting with Deep Learning

London households smart meter data

Thanks to my collaborator [Aaron Epel](http://www.linkedin.com/in/aaronepel/)!


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import IPython
import IPython.display
import glob
import time
import pickle
import sys
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings("ignore")
randomState = 42 # tip of the cap to Douglas Adams

# Load Data

## Load half-hourly electric usage data
...for ~5k smart meters in London
[SmartMeter Energy Consumption Data in London Households](https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households)

In [ ]:
# load half-hourly electric usage data
# takes about four minutes, need to find somerthing faster like Dask?
# https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households
# Get CSV files list from a folder
path = '/kaggle/input/smart-meters-in-london/halfhourly_dataset/halfhourly_dataset'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
start_time = time.time()
df_list = (pd.read_csv(file, parse_dates=["tstp"]) for file in csv_files)
print('%s seconds' % (time.time() - start_time))

# Concatenate all DataFrames
start_time = time.time()
d = pd.concat(df_list, ignore_index=True)
print('%s seconds' % (time.time() - start_time))

In [ ]:
d.describe()

## Load hourly weather data

In [ ]:
# load hourly weather data
# https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households
weatherData = pd.read_csv('/kaggle/input/smart-meters-in-london/weather_hourly_darksky.csv', parse_dates=["time"])

In [ ]:
weatherData.describe()

In [ ]:
weatherData.info()

# Data pre-processing and cleaning

## Weather data: convert text attributes to string datatype

In [ ]:
weatherData = weatherData.astype({'precipType':'string', 'icon':'string', 'summary':'string'})

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(weatherData, tsmode=True, sortby="time")
profile.to_file('weatherData profile_report.html')
# profile

## Identify and remove weather records not exactly on the hour

In [ ]:
# inspect and remove records not exactly on the hour
offRecs = weatherData.query("time.dt.minute != 0 or time.dt.second != 0")
print('Records not exactly on the half-hour:\n ', offRecs)

In [ ]:
# select weather data features of interest
weatherUpsample = weatherData[['time','temperature', 'dewPoint']].copy()
# weatherUpsample = weatherData[['time','temperature', 'dewPoint', 'pressure', 'humidity']].copy()
# pressure and humidity removed due to permutation feature importance results
weatherUpsample = weatherUpsample.sort_values(by=['time'])
print(weatherUpsample.info())
print(weatherUpsample.describe())
print(weatherUpsample)

## Upsample weather data to match half-houly sampling rate of load data

In [ ]:
# get the index set up to support the resamle operation
weatherUpsample.set_index('time', inplace=True)
weatherUpsample.index.rename('time', inplace=True)

start_time = time.time()
weatherUpsample = weatherUpsample.resample('30Min').mean()

# upsample
weatherUpsample['temperature'] = weatherUpsample['temperature'].interpolate()
weatherUpsample['dewPoint'] = weatherUpsample['dewPoint'].interpolate()
# weatherUpsample['pressure'] = weatherUpsample['pressure'].interpolate()
# weatherUpsample['humidity'] = weatherUpsample['humidity'].interpolate()

print('%s seconds' % (time.time() - start_time))

weatherUpsample = weatherUpsample.reset_index(names='DateTime')
print(weatherUpsample.info())
print(weatherUpsample.describe())
print(weatherUpsample)

In [ ]:
# save weather data so we don't have to do pre-processing again
weatherUpsample.to_csv('/kaggle/working/WeatherDataFinal.csv',index=False)

In [ ]:
# load pre-processed weather data
weatherUpsample = pd.read_csv('/kaggle/working/WeatherDataFinal.csv', parse_dates=["DateTime"])
weatherUpsample

In [ ]:
# utility function to nicely format variable names and memory they are consuming
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

## Convert smart meter usage datatype to float

In [ ]:
# ~1 minute
start_time = time.time()
d.iloc[:, 2] = pd.to_numeric(d.iloc[:, 2], errors='coerce')
print('%s seconds' % (time.time() - start_time))

# rename usage column for easier reference
d.rename(columns={d.columns[2]: 'kWhPerHalfHour'}, inplace=True)
d.info()

In [ ]:
# set timestamp as the index
start_time = time.time()
d.set_index('tstp')
print('%s seconds' % (time.time() - start_time))

## Identify and handle duplicates in the smart meter data

In [ ]:
# about 1.5 minutes
start_time = time.time()
dupes = d[d.duplicated()]
print('dupes', dupes)
print('dupes.index', dupes.index)
d.drop(index=dupes.index, inplace=True)
print('%s seconds' % (time.time() - start_time))

In [ ]:
# set index for the usage data to the timestamp column.  Is this necessary?  Can't remember why
start_time = time.time()
d.set_index('tstp')
d.info()
print('%s seconds' % (time.time() - start_time))

In [ ]:
# check what is gobbling RAM
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

# Exploratory Data Analysis
## Visualize smart meter dataset to analyze for quality, completenes and other insights

## grab a random sample of 2% of meters for visualization and analysis

In [ ]:
rng = np.random.default_rng(randomState)
# random_state = np.random.RandomState(randomState)
sampleMeters = rng.choice(d.LCLid.unique(), size=int(len(d.LCLid.unique())*0.02), replace=False)
print('sampleMeters:\n', sampleMeters)
sample = d[d['LCLid'].isin(sampleMeters)]
print('sample:\n', sample)
print(sample.describe())
# print(sample.info())

In [ ]:
sample.to_csv('/kaggle/working/sampleMeters.csv',index=False)

In [ ]:
sample = pd.read_csv('/kaggle/working/sampleMeters.csv', parse_dates=["tstp"])
sample

## Heatmap to visualize meter read coverage and completeness

In [ ]:
# visualize meter read coverage and completeness
# using a random sample of 2% of meters
import matplotlib.ticker as ticker
plt.subplots(figsize=(20,5))
pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='kWhPerHalfHour')
sns.heatmap(pivot_table, xticklabels=48*30)
plt.title('Meter Data Heatmap', size=15)
plt.savefig('meter data heatmap.png', format='png')

In [ ]:

'''
def to_datetime(x):
  return pd.to_datetime(x, format='%Y-%m-%d %H:%M')

def format_datetime(x):
  return x.strftime('%Y-%m-%d %H:%M')

# Create a pivot table of the data
# pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='kWhPerHalfHour')

# Convert the timestamps to datetime objects
pivot_table.columns = pivot_table.columns.map(to_datetime)

# Create a heatmap of the data
fig, ax = plt.subplots(figsize=(20, 5))
sns.heatmap(pivot_table, ax=ax, xticklabels=pivot_table.columns.map(format_datetime))

# Set the title and save the figure
plt.title('Meter Data Heatmap', size=15)
plt.savefig('meter data heatmap.png', format='png')
'''

## Identify and remove smart meter readings not exactly on the half-hour

In [ ]:
# identify and remove records not exactly on the half-hour
start_time = time.time()

offRecs = d.query("tstp.dt.minute not in (0,30) or tstp.dt.second != 0")
# aggLoad["DateTime"].dt.hour > 30
print('\nRecords not exactly on the half-hour:\n ', offRecs)
print(offRecs.info())

# delete records not exactly on the half-hour
d.drop(offRecs.index, inplace=True)

print('%s seconds' % (time.time() - start_time))

offRecs = d.query("tstp.dt.minute not in (0,30) or tstp.dt.second != 0")
print('\nRecords not exactly on the half-hour:\n ', offRecs)

In [ ]:
d.info()

In [ ]:
# check what is gobbling RAM
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

## Fill gaps in the smart meter data

Our heatmap above shows lots of gaps (small white vertical lines), and we'll fill those gaps using interpolation

### First step of filling these gaps is to create NaN records where records are missing

Then we can fill these gaps with interpolation

In [ ]:
# First step of interpolation is to create NaN records where records are missing
# about 2 minutes
d.sort_values(by=['tstp'], inplace=True)
d.set_index('tstp', inplace=True)
d.index.rename('tstp', inplace=True)

start_time = time.time()
# resample to create NaN records where records are missing
d = d.groupby('LCLid')\
                .resample('30Min')\
                .mean()

# fill the gaps with interpolation
d['kWhPerHalfHour'] = d['kWhPerHalfHour'].interpolate(limit=2, limit_area='inside')
d.reset_index(inplace=True)

print('%s seconds' % (time.time() - start_time))

## Check the meter data heatmap to see if gaps have been filled

In [ ]:
# visualize after interpolating missing values
d.info()
sample = d[d['LCLid'].isin(sampleMeters)]
pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='kWhPerHalfHour')
plt.subplots(figsize=(20,5))
sns.heatmap(pivot_table, xticklabels=48*30) # one xtick label every month
plt.title('Meter Data Heatmap', size=15)
plt.savefig('meter data heatmap gaps filled.png', format='png')

## Visualize zeros in the dataset using heatmap

I'm always curious to understand zeros in a dataset, and whether they are legitimate zero values, or indicate a data quality problem.

In [ ]:
# visualize zeros in the dataset
start_time = time.time()
sample = d[d['LCLid'].isin(sampleMeters)]
sample['ZerokWhPerHalfHour'] = sample['kWhPerHalfHour'] == 0
pivot_table = pd.pivot_table(sample, columns='tstp', index='LCLid', values='ZerokWhPerHalfHour')
print('%s seconds' % (time.time() - start_time))
plt.subplots(figsize=(20,5))
sns.heatmap(pivot_table, xticklabels=48*30)
plt.title('Meter data heatmap zero reads', size=15)
plt.savefig('meter data heatmap zeros.png', format='png')

In [ ]:
# take a snapshot of data
# takes about 11 minutes
start_time = time.time()
d.to_csv('/kaggle/working/MeterDataFinal.csv',index=False)
print('%s seconds' % (time.time() - start_time))

In [ ]:
# gather the meters from the sample that have zero reads and count how many zero reads each has
sampleMetersWithZeroReads = sample[sample['kWhPerHalfHour'] == 0].groupby('LCLid').agg('count')
sampleMetersWithZeroReads

In [ ]:
# investigate the meters with zero reads
MAC002050 = sample.query("LCLid == 'MAC002050'")
print(MAC002050)
fig, ax = plt.subplots(4,figsize=(20,9))

# plot whole ~2 years
ax[0].plot(MAC002050.tstp, MAC002050.kWhPerHalfHour)
ax[0].plot(MAC002050.tstp, MAC002050.ZerokWhPerHalfHour)
ax[0].set(ylabel='kWh per half-hour',
       title='Load from one Household MAC002050 with lots of zero values')
plt.tick_params(rotation=45)
ax[0].grid()

# zoom in
ax[1].plot(MAC002050.tstp[11000:15000], MAC002050.kWhPerHalfHour[11000:15000])
ax[1].plot(MAC002050.tstp[11000:15000], MAC002050.ZerokWhPerHalfHour[11000:15000])
ax[1].set(xlabel='time (s)', ylabel='kWh per half-hour')
plt.tick_params(rotation=45)
ax[1].grid()

# zoom in more...
ax[2].plot(MAC002050.tstp[13000:13500], MAC002050.kWhPerHalfHour[13000:13500])
ax[2].plot(MAC002050.tstp[13000:13500], MAC002050.ZerokWhPerHalfHour[13000:13500])
ax[2].set(xlabel='time (s)', ylabel='kWh per half-hour')
plt.tick_params(rotation=45)
ax[2].grid()

# zoom in to a different part of the series...
ax[3].plot(MAC002050.tstp[25000:25500], MAC002050.kWhPerHalfHour[25000:25500])
ax[3].plot(MAC002050.tstp[25000:25500], MAC002050.ZerokWhPerHalfHour[25000:25500])
ax[3].set(xlabel='time (s)', ylabel='kWh per half-hour')
plt.tick_params(rotation=45)
ax[3].grid()

fig.savefig("MAC002050.png")
plt.show()

Observation: The zeros for MAC002050 seem legit - leaving them in

In [ ]:
# investigate the meters with zero reads
MAC001558 = sample.query("LCLid == 'MAC001558'")
fig, ax = plt.subplots(4,figsize=(20,9))

# plot whole ~2 years
ax[0].plot(MAC001558.tstp, MAC001558.kWhPerHalfHour)
ax[0].plot(MAC001558.tstp, MAC001558.ZerokWhPerHalfHour)
ax[0].set(ylabel='kWh per half-hour',
       title='Load from one Household MAC001558 with lots of zero values')
plt.tick_params(rotation=45)
ax[0].grid()

# zoom in
ax[1].plot(MAC001558.tstp[17000:21000], MAC001558.kWhPerHalfHour[17000:21000])
ax[1].plot(MAC001558.tstp[17000:21000], MAC001558.ZerokWhPerHalfHour[17000:21000])
ax[1].set(xlabel='time (s)', ylabel='kWh per half-hour')
plt.tick_params(rotation=45)
ax[1].grid()

# zoom in more...
ax[2].plot(MAC001558.tstp[19300:19800], MAC001558.kWhPerHalfHour[19300:19800])
ax[2].plot(MAC001558.tstp[19300:19800], MAC001558.ZerokWhPerHalfHour[19300:19800])
ax[2].set(xlabel='time (s)', ylabel='kWh per half-hour')
plt.tick_params(rotation=45)
ax[2].grid()

# zoom in to a different part of the series...
ax[3].plot(MAC001558.tstp[25000:25500], MAC001558.kWhPerHalfHour[25000:25500])
ax[3].plot(MAC001558.tstp[25000:25500], MAC001558.ZerokWhPerHalfHour[25000:25500])
ax[3].set(xlabel='time (s)', ylabel='kWh per half-hour')
plt.tick_params(rotation=45)
ax[3].grid()

fig.savefig("MAC001558.png")
plt.show()

Observation: The zeros for MAC001558 seem legit - leaving them in

In [ ]:
# explore some basic stats for each house
print(d.groupby('LCLid').max().sort_values('tstp'))
print(d.groupby('LCLid').min().sort_values('tstp'))
print(d.groupby('LCLid').count().sort_values('tstp'))

print(d.groupby('LCLid').agg(['min', 'max', 'count']))


In [ ]:
# set index for the sample
sample.set_index('tstp')

## EDA: Visualize daily average load for each meter and all meters...

In [ ]:
# calculate average daily load profile for all meters...
# about 1.5 minutes

start_time = time.time()
avgLoadProfile = pd.DataFrame(d.groupby([d['tstp'].dt.hour, d['tstp'].dt.minute]).kWhPerHalfHour.mean())
avgLoadProfile = avgLoadProfile.reset_index(names=['hour', 'minute'])
avgLoadProfile['labels'] = pd.to_datetime(avgLoadProfile['hour'].astype(str) + ':' + avgLoadProfile['minute'].astype(str), format='%H:%M').dt.time
print('%s seconds' % (time.time() - start_time))

fig, ax = plt.subplots(figsize=(10,7))

ax.set_xticks(avgLoadProfile.index, avgLoadProfile.labels)

ax.set(xlabel='time (HH:MI)', ylabel='kWh per half-hour',
       title='Average Household 24 hour load profile')

# calculate average daily load for each meter...
start_time = time.time()
avgLoadProfileEachMeter = pd.DataFrame(d.groupby(['LCLid', d['tstp'].dt.hour, d['tstp'].dt.minute]).agg({'kWhPerHalfHour': 'mean'}))
avgLoadProfileEachMeter = avgLoadProfileEachMeter.reset_index(names=['LCLid', 'hour', 'minute'])
print('%s seconds' % (time.time() - start_time))
# print(avgLoadProfileEachMeter.info())
# print(avgLoadProfileEachMeter)

# plot every sample meter
for meter in sampleMeters:
    # print(meter)
    ax.plot(avgLoadProfileEachMeter.loc[avgLoadProfileEachMeter['LCLid'] == meter].index % 48,
            avgLoadProfileEachMeter.loc[avgLoadProfileEachMeter['LCLid'] == meter].kWhPerHalfHour,
           color='grey')

# plot the average
ax.plot(avgLoadProfile.index, avgLoadProfile.kWhPerHalfHour, linewidth=5)

plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Avg 24hr Load Profile every meter.png")
plt.show()

## Create aggregate features: aggregate load (our target) and count of meters

In [ ]:
# Calculate the sum of all loads and count of smart meters for each timestamp
start_time = time.time()
aggLoad = pd.DataFrame(d.groupby('tstp')['kWhPerHalfHour'].agg({'sum', 'count'}))
aggLoad.reset_index(inplace=True)
aggLoad.columns = ['tstp', 'numMeters', 'AggregateLoad']
print('%s seconds' % (time.time() - start_time))

print(aggLoad)
print(aggLoad.describe())
print(aggLoad.info())

In [ ]:
aggLoad.sort_values(by=['tstp'], inplace=True)
aggLoad.set_index('tstp', inplace=True)
aggLoad.index.rename('DateTimeIndex', inplace=True)
aggLoad.info()

In [ ]:
aggLoad['DateTime'] = aggLoad.index
aggLoad.info()

In [ ]:
# identify records with zero load
# start with the aggregated records with zero load
AggZeros = aggLoad.query("AggregateLoad == 0")
AggZeros

In [ ]:
# inspect and fix records not exactly on the half-hour
offRecs = aggLoad.query("DateTime.dt.minute not in (0,30) or DateTime.dt.second != 0")
print('Records not exactly on the half-hour: ', offRecs)
print(offRecs.info())

# delete records not exactly on the half-hour
aggLoad = aggLoad.drop(offRecs.index)

offRecs = aggLoad.query("DateTime.dt.minute not in (0,30) or DateTime.dt.second != 0")
print('Records not exactly on the half-hour: ', offRecs)

In [ ]:
# check the regularity of the observations (time between observations)
# print(pd.infer_freq(train_data.DateTime))
aggLoad.index.to_series().diff().value_counts()

In [ ]:
# Calculate moving average and stddev for the aggregated load across all meters
window_size = int(len(aggLoad.AggregateLoad) / 10)
print(window_size)

aggLoadMovingStdev = aggLoad.AggregateLoad.rolling(window_size).std()
aggLoadMovingStdev.columns = ['MovingStdev']

aggLoadMovingAvg = aggLoad.AggregateLoad.rolling(window_size).mean()
aggLoadMovingAvg.columns = ['MovingAvg']

print('aggLoadMovingStdev:\n', aggLoadMovingStdev)
print(aggLoadMovingStdev.info())
print('aggLoadMovingAvg:\n', aggLoadMovingAvg)
print(aggLoadMovingAvg.info())

In [ ]:
# Visualize aggregate load, moving average, moving standard deviation
# print(aggLoad)
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime, aggLoad.AggregateLoad, label="Aggregate Load")
ax.plot(aggLoad.DateTime, aggLoadMovingAvg, linewidth=3, label="Moving Average")
ax.plot(aggLoad.DateTime, aggLoadMovingStdev, linewidth=3, label="Moving Stdev")

ax.set(xlabel='time', ylabel='kWh per half-hour',
       title='Aggregate Household load 2012-2014')
plt.tick_params(rotation=45)
ax.grid()

plt.legend(fontsize=15)
fig.savefig("Aggregate Household load 2012-2014.png")
plt.show()

In [ ]:
# Show curve of number of meters contributing to the aggregate load
# This shows correlation of increased load with meters being added to the program during the recruitment period
# print(aggLoad)
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime, aggLoad.numMeters, linewidth=3, label="Number of Meters")

ax.set(xlabel='time', ylabel='Number of Meters',
       title='Meter growth 2012-2014')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Meter growth 2012-2014.png")
plt.legend(fontsize=15)
plt.show()

## Explore our aggregate load curve at various scales

Zoom in gradually to a single day

In [ ]:
# Aggregate Household load June-August
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime[10000:15000], aggLoad.AggregateLoad[10000:15000])

ax.set(xlabel='time (s)', ylabel='kWh per half-hour',
       title='Aggregate Household load June-August 2012')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load June-August 2012.png")
plt.show()

In [ ]:
# Aggregate Household load one month
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime[12000:13000], aggLoad.AggregateLoad[12000:13000])

ax.set(xlabel='time (s)', ylabel='kWh per half-hour',
       title='Aggregate Household load one month')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load one month.png")
plt.show()

In [ ]:
# Aggregate Household load ~two days
fig, ax = plt.subplots(figsize=(20,7))
ax.plot(aggLoad.DateTime[12500:12600], aggLoad.AggregateLoad[12500:12600])

ax.set(xlabel='time (s)', ylabel='kWh per half-hour',
       title='Aggregate Household load ~two days')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load two days.png")
plt.show()

In [ ]:
# Aggregate Household load (one day)
fig, ax = plt.subplots()
ax.plot(aggLoad.DateTime[12500:12550], aggLoad.AggregateLoad[12500:12550])

ax.set(xlabel='time (s)', ylabel='kWh per half-hour',
       title='Aggregate Household load (one day)')
plt.tick_params(rotation=45)
ax.grid()

fig.savefig("Aggregate Household load (one day).png")
plt.show()

In [ ]:
aggLoad.to_csv('/kaggle/working/aggLoadDataFinal.csv',index=False)

In [ ]:
aggLoad = pd.read_csv('/kaggle/working/aggLoadDataFinal.csv', parse_dates=["DateTime"])
aggLoad

In [ ]:
aggLoad.iloc[35299]

In [ ]:
# Join load data and weather data
mergeData = pd.merge(aggLoad, weatherUpsample, on='DateTime', copy=False)
print(mergeData.info())
mergeData

## Plot autocorrelation of aggregate load (target)

...to investigate cyclical properties

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Extract the AggregateLoad feature
aggregate_load = mergeData['AggregateLoad']

# Plot the autocorrelation plot
plot_acf(aggregate_load, lags=100)

# Show the plot
plt.show()

# Feature engineering

## Cyclical features

In [ ]:
# Add features useful for time series
print(mergeData.info())
# Cyclical features
# week of year
weekOfYear = mergeData.DateTime.dt.weekofyear
mergeData["weekOfYear_sin"] = np.sin(weekOfYear*(2.*np.pi/52))
mergeData["weekOfYear_cos"] = np.cos(weekOfYear*(2.*np.pi/52))
# day of week
dayOfWeek = mergeData.DateTime.dt.dayofweek
mergeData["dayOfWeek_sin"] = np.sin(dayOfWeek*(2.*np.pi/7))
mergeData["dayOfWeek_cos"] = np.cos(dayOfWeek*(2.*np.pi/7))
# day of year
# aggLoad["dayOfYear"] = aggLoad.DateTime.dt.dayofyear
# minute of the day
minuteOfDay = (mergeData.DateTime.dt.hour * 60) + mergeData.DateTime.dt.minute
mergeData["minuteOfDay_sin"] = np.sin(minuteOfDay*(2.*np.pi/48))
mergeData["minuteOfDay_cos"] = np.cos(minuteOfDay*(2.*np.pi/48))

## Decomposition features

In [ ]:
decomp_df = pd.DataFrame(mergeData.copy())
decomp_df = decomp_df.set_index(pd.DatetimeIndex(decomp_df['DateTime']))
decomp_df.index=decomp_df.DateTime
decomp_df = decomp_df.AggregateLoad
print(decomp_df.describe())
print(decomp_df.info)

In [ ]:
# Annual decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
yearly_seasonal_decomp = seasonal_decompose(mergeData['AggregateLoad'], period=17532)
mergeData['yearlySeasonal']=yearly_seasonal_decomp.seasonal
yearly_seasonal_decomp.plot()

In [ ]:
# daily decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
daily_seasonal_decomp = seasonal_decompose(mergeData['AggregateLoad'], period=48)
mergeData['dailyTrend']=daily_seasonal_decomp.trend
mergeData['dailySeasonal']=daily_seasonal_decomp.seasonal
mergeData['dailyResid']=daily_seasonal_decomp.resid
daily_seasonal_decomp.plot();

In [ ]:
# weekly decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
weekly_seasonal_decomp = seasonal_decompose(mergeData['AggregateLoad'], period=336)
mergeData['weeklyTrend']=weekly_seasonal_decomp.trend
mergeData['weeklySeasonal']=weekly_seasonal_decomp.seasonal
mergeData['weeklyResid']=weekly_seasonal_decomp.resid
weekly_seasonal_decomp.plot();

## Lag features

In [ ]:
# load 1 day lag
oneDayPeriods = 48
mergeData['AggregateLoad_1dayLag'] = mergeData['AggregateLoad'].shift(oneDayPeriods)
# load 1 week lag
oneWeekPeriods = oneDayPeriods * 7
mergeData['AggregateLoad_1weekLag'] = mergeData['AggregateLoad'].shift(oneWeekPeriods)

# load change from last half-hour to this half-hour
mergeData['AggregateLoad_halfhourdiff'] = mergeData['AggregateLoad'] - mergeData['AggregateLoad'].shift(1)
# load change from one week ago to this half-hour
mergeData['AggregateLoad_weekdiff'] = mergeData['AggregateLoad'] - mergeData['AggregateLoad'].shift(oneWeekPeriods)

print(mergeData.info())

In [ ]:
# drop NaNs created by feature engineering
mergeData.dropna(inplace=True)

In [ ]:
# temperature change from last half-hour to this half-hour
# eliminated due to permutation feature importance
# weatherUpsample['temp_halfhourdiff'] = weatherUpsample['temperature'] - weatherUpsample['temperature'].shift(1)

# max temp for the day
mergeData['temp_daymax'] = mergeData.groupby(mergeData.DateTime.dt.date)['temperature'].transform('max')
mergeData['temp_daymin'] = mergeData.groupby(weatherUpsample.DateTime.dt.date)['temperature'].transform('min')
print(mergeData['temp_daymax'])
print(mergeData['temp_daymin'])

In [ ]:
"""
# Find the earliest date when there is the maximum number of meters contributing to the aggregate load
# will disreard all data beofre this point
maxMeters = aggLoad['numMeters'].max()
print(maxMeters)
startDateTime = aggLoad[aggLoad['numMeters']==maxMeters].DateTime.min()
print(startDateTime)
startDate = startDateTime.date()
print(startDate)
"""

In [ ]:
# Move first column to the Last
# df = pd.DataFrame(mergeData)
df = mergeData
temp_cols=df.columns.tolist()
new_cols=temp_cols[1:] + temp_cols[0:1]
mergeData=df[new_cols]
print(mergeData)

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(mergeData, tsmode=True, sortby="DateTime")
profile.to_file('mergeData profile_report.html')

In [ ]:
# Remove the dateTime feature from the dataset (we've extracted to features we need from it)
print(mergeData)
mergeData.drop(columns=['DateTime'], inplace=True)

In [ ]:
mergeData.to_csv('/kaggle/working/mergeDataFeatureCandidates.csv',index=False)

In [ ]:
mergeData = pd.read_csv('/kaggle/working/mergeDataFeatureCandidates.csv')
mergeData

## Final feature selection

In [ ]:
# select features
mergeData = mergeData[['weekOfYear_sin', 'weekOfYear_cos', 'dayOfWeek_sin', 'dayOfWeek_cos', 'dailyTrend', 'dailySeasonal', 'dailyResid', 'weeklyTrend', 'weeklySeasonal', 'weeklyResid', 'AggregateLoad_1dayLag', 'AggregateLoad_1weekLag', 'AggregateLoad_halfhourdiff', 'temperature', 'temp_daymax', 'AggregateLoad']]
mergeData.info()

# mergeData = mergeData[['AggregateLoad_1dayLag', 'AggregateLoad_1weekLag', 'dayOfWeek', 'temperature','AggregateLoad']]
# mergeData = mergeData[['numMeters', 'temperature', 'minuteOfDay_sin', 'minuteOfDay_cos', 'dayOfWeek_sin', 'dayOfWeek_cos', 'weekOfYear_sin', 'weekOfYear_cos', 'AggregateLoad']]
# mergeData = mergeData[['numMeters', 'minuteOfDay_sin', 'minuteOfDay_cos', 'dayOfWeek_sin', 'dayOfWeek_cos', 'weekOfYear_sin', 'weekOfYear_cos', 'AggregateLoad']]
# mergeData = mergeData[['minuteOfDay_sin', 'minuteOfDay_cos', 'dayOfWeek_sin', 'dayOfWeek_cos', 'weekOfYear_sin', 'weekOfYear_cos', 'AggregateLoad']]
# mergeData = mergeData[['AggregateLoad']]
# remove features dues to feature permutation importance
# mergeData.drop(columns=['numMeters', 'yearlySeasonal'], inplace=True)
# mergeData.drop(columns=['temperature', 'AggregateLoad_1weekLag', 'temp_daymax'], inplace=True)
# after looking at correlation matrix and hearing about overalp between cyclical encoding and seasinal decomp...
# mergeData.drop(columns=['dewPoint', 'dayOfWeek_sin', 'minuteOfDay_sin', 'minuteOfDay_cos', 'weeklyTrend', 'weeklySeasonal', 'weeklyResid', 'temp_daymin', 'AggregateLoad_1dayLag'], inplace=True)
# ok have to put those cyclical features back in...
# mergeData.drop(columns=['dewPoint', 'weeklyTrend', 'weeklySeasonal', 'weeklyResid', 'temp_daymin', 'AggregateLoad_1dayLag'], inplace=True)
# ok now drop these after feature importance...
# mergeData.drop(columns=['yearlySeasonal', 'numMeters', 'minuteOfDay_sin', 'minuteOfDay_cos', 'AggregateLoad_weekdiff'], inplace=True)

# going back to best feature set


In [ ]:
mergeData.to_csv('/kaggle/working/mergeDataFinal.csv',index=False)

In [ ]:
mergeData = pd.read_csv('/kaggle/working/mergeDataFinal.csv')
mergeData

## Training, Validation, Testing Split

In [ ]:
# Split the time series data into train, test, and validation datasets
train_size = int(len(mergeData) * 0.7)  # 70% for training
val_size = int(len(mergeData) * 0.2)   # 20% for validation
test_size = len(mergeData) - val_size - train_size  # Remaining 10% for testing

train_data = mergeData[:train_size].copy()
train_data.reset_index(drop=True, inplace=True)
val_data = mergeData[train_size:train_size+val_size].copy()
val_data.reset_index(drop=True, inplace=True)
test_data = mergeData[train_size+val_size:].copy()
test_data.reset_index(drop=True, inplace=True)

print('\ntrain_data.head()\n', train_data.head())
print(train_data.info())
print('\nval_data.head()\n', val_data.head())
print(val_data.info())
print('\ntest_data.head()\n', test_data.head())
print(test_data.info())

num_out_features = mergeData.shape[1]
label_columns = ['AggregateLoad']

In [ ]:
def prediction_plot(testY, test_predict):
      len_prediction=[x for x in range(len(testY))]
      plt.figure(figsize=(20,5))
      plt.plot(len_prediction, testY, marker='.', label="actual")
      plt.plot(len_prediction, test_predict, 'r', label="prediction")
      plt.tight_layout()
      sns.despine(top=True)
      plt.subplots_adjust(left=0.07)
      plt.ylabel('kWh per half hour', size=15)
      plt.xlabel('Time step', size=15)
      plt.legend(fontsize=15)
      plt.show();

## Standardize the data

In [ ]:
# Standardize the data
train_mean = train_data.mean()
train_std = train_data.std()

train_data = (train_data - train_mean) / train_std
val_data = (val_data - train_mean) / train_std
test_data = (test_data - train_mean) / train_std

mergeDataNormed = (mergeData - train_mean) / train_std

In [ ]:
train_data.to_csv('/kaggle/working/train_data.csv',index=False)
val_data.to_csv('/kaggle/working/val_data.csv',index=False)
test_data.to_csv('/kaggle/working/test_data.csv',index=False)
mergeDataNormed.to_csv('/kaggle/working/mergeDataNormed.csv',index=False)

In [ ]:
train_data = pd.read_csv('/kaggle/working/train_data.csv')
val_data = pd.read_csv('/kaggle/working/val_data.csv')
test_data = pd.read_csv('/kaggle/working/test_data.csv')
mergeDataNormed = pd.read_csv('/kaggle/working/mergeDataNormed.csv')
label_columns = ['AggregateLoad']

In [ ]:
print(train_data.info())
print(val_data.info())
print(test_data.info())
print(mergeDataNormed.info())

In [ ]:
# Visualize distribution of the features
# df_std = (mergeData - train_mean) / train_std
df_std = mergeDataNormed.melt(var_name='Column', value_name='Standardized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Standardized', data=df_std, size=15)
_ = ax.set_xticklabels(mergeDataNormed.keys(), rotation=90)
plt.title('Violin Plot', size=15)
plt.savefig('violin_plot.png', format='png')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mutual_info_score

# Split the data into features and target
X = mergeDataNormed.drop(columns=['AggregateLoad'])
y = mergeDataNormed['AggregateLoad']

# Calculate mutual information between each feature and the target variable
mutual_information_scores = []
for feature in X.columns:
    mutual_information_score = mutual_info_score(X[feature], y)
    mutual_information_scores.append([feature,mutual_information_score])

MISdf = pd.DataFrame(mutual_information_scores, columns=['Feature', 'Mutual information score'])
# Print the mutual information scores
# print('Mutual information scores:', mutual_information_scores)
print(MISdf.sort_values('Mutual information score', ascending=False))


In [ ]:
from pandas_profiling import ProfileReport

# profile = ProfileReport(mergeDataNormed, tsmode=True, sortby=mergeDataNormed.index.astype(int))
profile = ProfileReport(mergeDataNormed, tsmode=True)
profile.to_file('mergeDataNormed profile_report.html')
# profile

## Functions for preparing data for time series machine learning

Not my work

Credit to [Tensorflow Tutorial: Time series forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series)

In [ ]:
class WindowGenerator():
    # https://www.tensorflow.org/tutorials/structured_data/time_series#1_indexes_and_offsets
  def __init__(self, input_width, label_width, shift,
               train_df=train_data, val_df=val_data, test_df=test_data,
               label_columns=None):
    # print('\nWindowGenerator.__init__\n')
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    # print('\nlabel_columns:\n', label_columns)
    # Work out the label column indices.
    self.label_columns = label_columns
    self.num_out_features = train_df.shape[1] # default to predicting all input columns

    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
      self.num_out_features = len(label_columns) # JS added this
      # print('\nself.num_out_features:', self.num_out_features)

    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
    # https://www.tensorflow.org/tutorials/structured_data/time_series#2_split
  # print('\nsplit_window\n', features)
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window


In [ ]:
def plot(self, model=None, plot_col='AggregateLoad', max_subplots=3):
    # https://www.tensorflow.org/tutorials/structured_data/time_series#3_plot
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
def make_dataset(self, data):
    # https://www.tensorflow.org/tutorials/structured_data/time_series#4_create_tfdatadatasets
  # print('\nmake_dataset\n', data)
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      seed=randomState,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
# https://www.tensorflow.org/tutorials/structured_data/time_series#4_create_tfdatadatasets
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

## Create data windows for time series forecasting

In [ ]:
# Prepare data for one-shot multi-step
OUT_STEPS = 48 # 24 hour forecast
# IN_STEPS = 336 # look back 1 week
IN_STEPS = 48 # look back 1 day
multi_window = WindowGenerator(input_width=IN_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=['AggregateLoad'],
                              train_df=train_data, val_df=val_data, test_df=test_data,)

num_out_features = multi_window.num_out_features
# print('\nnum_out_features: ', num_out_features)
multi_window.plot()
multi_window

# Build and Train models

## Baseline persistence model

Use a 1 week naive persistence model as a baseline to evaluate performance of the machine learning models we are building

Georgios Tziolis, Chrysovalantis Spanias, Maria Theodoride, Spyros Theocharides, Javier Lopez-Lorente, Andreas Livera, George Makrides, George E. Georghiou,

Short-term electric net load forecasting for solar-integrated distribution systems based on Bayesian neural networks and statistical post-processing,

Energy,
Volume 271,
2023,
127018,
ISSN 0360-5442,

https://doi.org/10.1016/j.energy.2023.127018.

In [ ]:
# capture performnce of models
multi_val_performance = {}
multi_test_performance = {}

### Function to compile and fit models

In [ ]:
MAX_EPOCHS = 100

def compile_and_fit(model, window, patience=5):
  # https://www.tensorflow.org/tutorials/structured_data/time_series#linear_model
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  checkpoint_filepath = '/tmp/' + model.name + '/checkpoint'
  checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath,
                    monitor="val_loss", mode="min",
                    save_best_only=True, verbose=1)

  # print('\nwindow.train:\n', window.train)
  # print('\nwindow.val:\n', window.val)
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, checkpoint])

  # restore weights from epoch with best loss against validation dataset
  model.load_weights(checkpoint_filepath)

  return history


In [ ]:
# Baseline Model: Naive 1 week persistence
OneWeekNPeriods = 48 * 7
NaiveForecast = mergeData.AggregateLoad.shift(OneWeekNPeriods).copy()

print(NaiveForecast.info())
print(NaiveForecast.describe())

In [ ]:
# visualize naive forecast and actuals for entire dataset
prediction_plot(mergeData.AggregateLoad, NaiveForecast)

In [ ]:
# visualize naive forecast and actuals for first 24 hours of the test dataset
# print(train_size, val_size, test_size)
# print(train_size+val_size)
prediction_plot(mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], NaiveForecast[train_size+val_size:train_size+val_size+OUT_STEPS])

In [ ]:
# standardize the baseline
NaiveForecastNormed = NaiveForecast.transform(lambda x: (x - train_mean) / train_std)
NaiveForecastNormed.describe()

In [ ]:
# calculate error for baseline naive model (Normed)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# calculate error for naive model on validation set
valNaiveMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size:train_size+val_size], NaiveForecastNormed.AggregateLoad[train_size:train_size+val_size])

# calculate error for naive model on test set
testNaiveMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size+val_size:], NaiveForecastNormed.AggregateLoad[train_size+val_size:])

print('valNaiveMAE: ', valNaiveMAE)
print('testNaiveMAE: ', testNaiveMAE)

In [ ]:
# calculate error for baseline naive model (not normed)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
print('Naive Root Mean Squared Error(RMSE): %.2f; Naive Mean Absolute Error(MAE) : %.2f; Naive Mean Absolute Percantage Error(MAPE) : %.2f '
      % (np.sqrt(mean_squared_error(mergeData.AggregateLoad[OneWeekNPeriods:], NaiveForecast[OneWeekNPeriods:])),
         mean_absolute_error(mergeData.AggregateLoad[OneWeekNPeriods:], NaiveForecast[OneWeekNPeriods:]),
         mean_absolute_percentage_error(mergeData.AggregateLoad[OneWeekNPeriods:], NaiveForecast[OneWeekNPeriods:])))

# calculate error for naive model on validation set
valNaiveMAE = mean_absolute_error(mergeData.AggregateLoad[train_size:train_size+val_size], NaiveForecast[train_size:train_size+val_size])

# calculate error for naive model on test set
testNaiveMAE = mean_absolute_error(mergeData.AggregateLoad[train_size+val_size:], NaiveForecast[train_size+val_size:])

print('valNaiveMAE: ', valNaiveMAE)
print('testNaiveMAE: ', testNaiveMAE)


In [ ]:
# function for plotting the train and test loss curves
def plot_model_loss(history):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.show()
    return

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                        metrics=[tf.keras.metrics.MeanAbsoluteError()])

multi_val_performance['Baseline'] = repeat_baseline.evaluate(mergeDataNormed.AggregateLoad[train_size:train_size+val_size], NaiveForecastNormed.AggregateLoad[train_size:train_size+val_size], verbose=0)
multi_test_performance['Baseline'] = repeat_baseline.evaluate(mergeDataNormed.AggregateLoad[train_size+val_size:], NaiveForecastNormed.AggregateLoad[train_size+val_size:], verbose=0)
print(multi_val_performance['Baseline'], multi_test_performance['Baseline'])

## Multi Layer Perceptron (MLP) Model

In [ ]:
multi_MLP_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dense(32, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_out_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_out_features])
], name="MLP")

history = compile_and_fit(multi_MLP_model, multi_window)
multi_MLP_model.save('multi_MLP_model.keras')
plot_model_loss(history)

multi_val_performance['MLP'] = multi_MLP_model.evaluate(multi_window.val)
multi_test_performance['MLP'] = multi_MLP_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_MLP_model)

In [ ]:
num_out_features

## Convolutional Neural Network (CNN) Model



In [ ]:
CONV_WIDTH = 10
multi_cnn_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(64, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    # tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(OUT_STEPS*num_out_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_out_features])
], name="CNN")

history = compile_and_fit(multi_cnn_model, multi_window)
multi_cnn_model.save('multi_cnn_model.keras')
plot_model_loss(history)
# IPython.display.clear_output()

multi_val_performance['CNN'] = multi_cnn_model.evaluate(multi_window.val)
multi_test_performance['CNN'] = multi_cnn_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_cnn_model)


## Recurrent Neural Network (RNN) Model

In [ ]:
multi_rnn_model = tf.keras.Sequential([
    # tf.keras.layers.SimpleRNN(32, return_sequences=True),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.SimpleRNN(64, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*num_out_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_out_features])
], name="RNN")

history = compile_and_fit(multi_rnn_model, multi_window)
plot_model_loss(history)
multi_rnn_model.save('multi_rnn_model.keras')
# IPython.display.clear_output()

multi_val_performance['RNN'] = multi_rnn_model.evaluate(multi_window.val)
multi_test_performance['RNN'] = multi_rnn_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_rnn_model)

## Long Short Term Memory (LSTM) Model

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # tf.keras.layers.LSTM(32,return_sequences=True),
    tf.keras.layers.LSTM(64,return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_out_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_out_features])
], name="LSTM")

start_time = time.time()
history = compile_and_fit(multi_lstm_model, multi_window)
print('%s seconds' % (time.time() - start_time))
multi_lstm_model.save('multi_lstm_model.keras')
plot_model_loss(history)
# IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_test_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)


## GRU Model

In [ ]:
multi_gru_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    # tf.keras.layers.GRU(32, return_sequences=True),
    tf.keras.layers.GRU(64, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_out_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_out_features])
], name="GRU")

start_time = time.time()
history = compile_and_fit(multi_gru_model, multi_window)
print('%s seconds' % (time.time() - start_time))
multi_gru_model.save('multi_gru_model.keras')
plot_model_loss(history)
# IPython.display.clear_output()

multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val, verbose=0)
multi_test_performance['GRU'] = multi_gru_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_gru_model)

# Model Evaluation and Selection

print('\nValidation Performance:\n', multi_val_performance)
print('\nTest Performance:\n', multi_test_performance)

In [ ]:
x = np.arange(len(multi_test_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = multi_lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_test_performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_test_performance.keys(),
           rotation=45)
plt.ylabel(f'Mean Absolute Error (MAE)')
_ = plt.legend()
plt.title(f'Model Performance')
plt.savefig('model performance.png', format='png')


In [ ]:
CONV_WIDTH = 10 # CNN model would not load without this...
# restore models from file
multi_MLP_model = tf.keras.models.load_model('multi_MLP_model.keras')
multi_cnn_model = tf.keras.models.load_model('multi_cnn_model.keras')
multi_rnn_model = tf.keras.models.load_model('multi_rnn_model.keras')
multi_lstm_model = tf.keras.models.load_model('multi_lstm_model.keras')
multi_gru_model = tf.keras.models.load_model('multi_gru_model.keras')
model_list = [multi_MLP_model, multi_cnn_model, multi_rnn_model, multi_lstm_model, multi_gru_model]


In [ ]:
for model in model_list:
    print(model.name)

In [ ]:
# store the evaluation results for all models
import pickle
with open('model eval results.pkl', 'wb') as f:
    pickle.dump(multi_val_performance, f)

with open('model test results.pkl', 'wb') as f:
    pickle.dump(multi_test_performance, f)

In [ ]:
# load the evaluation results for all models
with open('model eval results.pkl', 'rb') as f:
    multi_val_performance = pickle.load(f)

with open('model test results.pkl', 'rb') as f:
    multi_test_performance = pickle.load(f)

print(multi_val_performance)
print(multi_test_performance)

## Feature Evaluation (methods that require models already built)

In [ ]:
# function to create time series dataset for feature permutation
def generateFeatPermWindow(input_width=IN_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=None,
                                cols_to_shuffle=None,
                              train_df=train_data, val_df=val_data, test_df=test_data):

    train = train_df.copy()
    val = val_df.copy()
    test = test_df.copy()

    # randomly shuffle shuffle the specified columns
    for col in cols_to_shuffle:
        train[col] = train[col].sample(frac=1, replace=False, random_state=randomState, ignore_index=True)
        val[col] = val[col].sample(frac=1, replace=False, random_state=randomState, ignore_index=True)
        test[col] = test[col].sample(frac=1, replace=False, random_state=randomState, ignore_index=True)

    """
    print('train NaNs: ', train.isnull().sum().sum())
    print('val NaNs: ', val.isnull().sum().sum())
    print('val NaNs:\n', val[val.isna().any(axis=1)])
    print('test NaNs: ', test.isnull().sum().sum())
    print('test NaNs:\n', test[test.isna().any(axis=1)])
    """

    feat_perm_window = WindowGenerator(input_width=IN_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=label_columns,
                              train_df=train, val_df=val, test_df=test,)

    return feat_perm_window


In [ ]:
import numpy as np
import pandas as pd
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Define a function to calculate the permutation importance of a feature group for a given model
def calculate_permutation_importance_group(model, feature_group, baseline_val_perf, baseline_test_perf):
    """Calculates the permutation importance of a feature group for a given model.

    Args:
        model: The model to calculate the permutation importance for.
        feature_group: A list of the features in the feature group.
        baseline_val_perf: baseline performance (in MAE) of the model using all features for the validation set
        baseline_test_perf: baseline performance (in MAE) of the model using all features for the test set

    Returns:
        The permutation importance of the feature group for the given model.
    """

    # Shuffle the values of the features in the feature group
    feat_perm_window = generateFeatPermWindow(input_width=IN_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=label_columns,
                                cols_to_shuffle=feature_group,
                              train_df=train_data, val_df=val_data, test_df=test_data)

    # model.fit(X_shuffled, y)
    val_perf_shuffle = model.evaluate(feat_perm_window.val, verbose=1)[1] # MAE

    # Calculate the permutation importance
    # importance = np.mean(y - y_pred)
    importance = val_perf_shuffle - baseline_val_perf

    print('\nModel: ', model.name, feature_group, 'importance: ', importance)

    return importance

In [ ]:
# Define a function to visualize the permutation importances for a given feature using a stacked bar chart
def visualize_permutation_importances_stacked_bar_stackplot(model, permutation_importances):
    """Visualizes the permutation importances for a given feature using a stacked bar chart created with the plt.stackplot() function.

    Args:
        feature: The feature to visualize the permutation importances for.
        permutation_importances: A dictionary mapping the model names to the permutation importances for the given feature.
    """

    # Get the permutation importances for each feature in a list
    importances = sorted(permutation_importances.items(), key=itemgetter(1), reverse=True)
    permutation_importances_values = list(importances.values())

    # Create a bar chart of the permutation importances
    plt.bar(importances.keys(), permutation_importances_values)
    plt.xlabel('Feature')
    plt.ylabel('Permutation Importance')
    plt.title('Permutation Importances for Model: {}'.format(model))
    plt.legend(importances.keys())
    plt.tick_params(rotation=45)
    plt.show()

## Permutation Feature Importance

In [ ]:
# Calculate the permutation feature importance for each model and visualize the results
# cyclically encoded features need to be grouped together for the purposes of feature permutation importance
feature_groups = {
    'weekOfYear': ['weekOfYear_sin', 'weekOfYear_cos'],
    'dayOfWeek': ['dayOfWeek_sin', 'dayOfWeek_cos'],
#    'minuteOfDay': ['minuteOfDay_sin', 'minuteOfDay_cos'],
}

grouped_features = [value for key, value in feature_groups.items() for value in value]
print('grouped_features: ', grouped_features, '\n')

# add individual features as feature groups of one
for feature in val_data.columns:
    if feature not in grouped_features:
        feature_groups[feature] = [feature]

print('feature_groups: ', feature_groups)

permutation_importances_groups = {}
for model in model_list:
    # calculate baseline model performance with all features
    print('\ncalculate baseline model performance with all features.\n', model.name)
    val_perf = model.evaluate(multi_window.val, verbose=1)[1] # MAE
    test_perf = model.evaluate(multi_window.test, verbose=1)[1] # MAE

    for feature_group_name, feature_group in feature_groups.items():
        permutation_importance = calculate_permutation_importance_group(model, feature_group, val_perf, test_perf)
        if model.name not in permutation_importances_groups:
            permutation_importances_groups[model.name] = {}
        permutation_importances_groups[model.name][feature_group_name] = permutation_importance


In [ ]:
permutation_importances_groups

In [ ]:
from operator import itemgetter, attrgetter
# Define a function to visualize the permutation importances for a given feature using a stacked bar chart
def visualize_permutation_importances_stacked_bar_stackplot(model, permutation_importances):
    """Visualizes the permutation importances for a given feature using a stacked bar chart created with the plt.stackplot() function.

    Args:
        feature: The feature to visualize the permutation importances for.
        permutation_importances: A dictionary mapping the model names to the permutation importances for the given feature.
    """

    # Get the permutation importances for each feature in a list
    importances = sorted(permutation_importances.items(), key=itemgetter(1), reverse=True)
    print(importances)
    df = pd.DataFrame(importances, columns=['label', 'value'])
    labels = df['label'].tolist()
    values = df['value'].tolist()


    # Create a stacked bar chart of the permutation importances
    plt.bar(labels, values)
    plt.xlabel('Feature')
    plt.ylabel('Permutation Importance')
    plt.title('Permutation Importances for Model: {}'.format(model))
    plt.legend(labels)
    plt.tick_params(rotation=90)
    plt.show()

In [ ]:
# Visualize the permutation importances for each feature using a stacked bar chart
for model, permutation_importances in permutation_importances_groups.items():
    visualize_permutation_importances_stacked_bar_stackplot(model, permutation_importances)

# Best Model Results

## Single Day results: Benchmark, GRU model and actuals

In [ ]:
# build a single record dataset from start of the test dataset
# print(mergeDataNormed)
# print(train_size+test_size-IN_STEPS)
# print(mergeDataNormed.iloc[train_size+test_size-IN_STEPS:train_size+test_size])

oneDay_ds = tf.keras.utils.timeseries_dataset_from_array(
      data=test_data[:IN_STEPS],
      targets=None,
      sequence_length=IN_STEPS,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)

# get the prediction
testYhatNormed = multi_gru_model.predict(oneDay_ds)

In [ ]:
# print(testYhatNormed) # prediction for all input columns
print(testYhatNormed[0,:,-1]) # prediction for target column, aggregate load

In [ ]:
# Invert standardization
testYhat = (np.array(testYhatNormed) * np.array(train_std)) + np.array(train_mean)

# print(testYhat)
print(testYhat[0,:,-1])

In [ ]:
# Plot Naive prediction, GRU prediction and Actuals for first 24 hours of the test set (normed values)
len_prediction=[x for x in range(len(testYhat[0,:,-1]))]
plt.figure(figsize=(10,5))
# plt.plot(len_prediction, test_data[:OUT_STEPS].AggregateLoad, marker='.', label="actual")
plt.plot(len_prediction, mergeDataNormed.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], marker='.', label="Actual")
plt.plot(len_prediction, testYhatNormed[0,:,-1], 'r', label="GRU prediction")
plt.plot(len_prediction, NaiveForecastNormed[train_size+val_size:train_size+val_size+OUT_STEPS].AggregateLoad, 'g', label="Baseline prediction")

plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('kWh per half hour (Standardized)', size=15)
plt.xlabel('Time (half-hours)', size=15)
plt.legend(fontsize=15)
plt.title('Baseline, Prediction and Actuals (Standardized)', size=15)
plt.show();
plt.savefig('Baseline, Prediction and Actuals (Standardized).png', format='png')

In [ ]:
# calculate error for baseline naive model (Normed) first 24 hours of test set
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# calculate error for baseline naive model on validation set
valBaselineMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size:train_size+OUT_STEPS], NaiveForecastNormed.AggregateLoad[train_size:train_size+OUT_STEPS])

# calculate error for baseline naive model on test set
testBaselineMAE = mean_absolute_error(mergeDataNormed.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], NaiveForecastNormed.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS])

print('valBaselineMAE: ', valBaselineMAE)
print('testBaselineMAE: ', testBaselineMAE)

In [ ]:
len_prediction=[x for x in range(len(testYhat[0,:,-1]))]
plt.figure(figsize=(10,5))
# plt.plot(len_prediction, test_data[:OUT_STEPS].AggregateLoad, marker='.', label="actual")
plt.plot(len_prediction, mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], marker='.', label="Actual")
plt.plot(len_prediction, testYhat[0,:,-1], 'r', label="GRU prediction")
plt.plot(len_prediction, NaiveForecast[train_size+val_size:train_size+val_size+OUT_STEPS], 'g', label="Baseline prediction")

plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('kWh per half hour', size=15)
plt.xlabel('Hour', size=15)
plt.legend(fontsize=15)
plt.title('Baseline, Prediction and Actuals', size=15)
plt.show()
plt.savefig('Baseline, Prediction and Actuals.png', format='png')

In [ ]:
# calculate error for baseline naive model first 24 hours of test set
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# calculate error for baseline naive model on test set
testBaselineMAE1day = mean_absolute_error(mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], NaiveForecast[train_size+val_size:train_size+val_size+OUT_STEPS])

# calculate error for baseline naive model on test set
testModelMAE1day = mean_absolute_error(mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS], testYhat[0,:,-1])

print('testBaselineMAE1day: ', testBaselineMAE1day)
print('testModelMAE1day: ', testModelMAE1day)
print('Model accuracy absolute improvement: ', testBaselineMAE1day - testModelMAE1day)
print('Model accuracy % improvement: ', (testBaselineMAE1day - testModelMAE1day) / testBaselineMAE1day * 100)

## Aggregate results for Test period: Benchmark, GRU model and actuals

In [ ]:
# build dataset for the test dataset

ds = tf.keras.utils.timeseries_dataset_from_array(
      data=test_data,
      targets=None,
      sequence_length=IN_STEPS,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)

# get the prediction
testYhatNormed = multi_gru_model.predict(ds)

In [ ]:
print(testYhatNormed.shape)
print(testYhatNormed)

In [ ]:
# Invert standardization
testYhat = testYhatNormed * train_std.AggregateLoad + train_mean.AggregateLoad

print(testYhat.shape)
print(testYhat)

# Estimate financial impact of improved forecast

## Load pricing data

Load day forward and real time wholesale electricity pricing data to help us estimate financial benefit of the best model.

In [ ]:
# https://www.nordpoolgroup.com/en/Market-data1/Intraday/intraday-auction-uk/uk/evening-auction-17.30-bst/prices-and-volumes/half-hour/?view=table
# https://www.nordpoolgroup.com/en/Market-data1/GB/Auction-prices/no2/hourly/?view=table
# prices = pd.read_csv('/kaggle/input/uk-elec-prices/Elec price curves - UK Price Curves multiple days.csv', parse_dates=['Date'])
prices = pd.read_csv('/kaggle/input/uk-elec-prices/Elec price curves - UK price curves 24-11-2-2022.csv')
prices

## Visualize pricing curve

In [ ]:
# Plot Naive prediction, GRU prediction and Actuals for first 24 hours of the test set (normed values)
len_prediction=[x for x in range(len(prices['Real Time Price']))]
plt.figure(figsize=(10,5))
plt.plot(len_prediction, prices['Day Ahead Price'], label="Day Ahead Price")
plt.plot(len_prediction, prices['Real Time Price'], 'r', label="Real Time Price")

plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Price (GBP/MWH)', size=15)
plt.xlabel('Hour', size=15)
plt.legend(fontsize=15)
plt.title('UK Price Curve Analog: 11-24-2022', size=15)
plt.show();
plt.savefig('Price Curve.png', format='png')

## Calculate baseline estimate overestimate and underestimate profile




In [ ]:
import math

def calculate_forecast_penalty(forecast, actual, prices):
    # forecast is a list of 48 half-hourly forecast values, in kWh
    # actual is a list of 48 half-hourly actual values, in kWh
    # prices is a dataframe with 24 day-ahead and real-time wholesale electricity prices, in $/MWh

    misEstDollars = []
    dollarPenalty = 0
    counter = 0

    misEstimate = forecast - actual

    for x in misEstimate:
        priceIndex = math.floor(counter / 2) # we only have hourly pricing data
        # print('priceIndex: ', priceIndex)
        if x > 0:
            # overestimate
            dollarPenalty = x * prices.iloc[priceIndex]['Day Ahead Price']
        elif x < 0:
            # underestimate
            # dollarPenalty = -x * (prices.iloc[priceIndex]['Real Time Price'] - prices.iloc[priceIndex]['Day Ahead Price'])
            dollarPenalty = -x * (prices.iloc[priceIndex]['Real Time Price'])
        else:
            dollarPenalty = 0
        misEstDollars.append(dollarPenalty / 1000) # convert MWh to kWh
        counter += 1

    print('\nDollar penalty for misestimation each half-hour: ', misEstDollars)
    TotalDollarPenalty = sum(misEstDollars)
    print('\nTotal dollar penalty for misestimation for the day: ', TotalDollarPenalty)
    return TotalDollarPenalty, misEstDollars


In [ ]:
# calculate financial penalty for baseline
baselinePenalty, baselinePenaltyHourly = calculate_forecast_penalty(forecast=NaiveForecast[train_size+val_size:train_size+val_size+OUT_STEPS],
                           actual=mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS],
                           prices=prices)
baselinePenalty

In [ ]:
# calculate financial penalty for model
modelPenalty, modelPenaltyHourly = calculate_forecast_penalty(forecast=testYhat[0,:,-1],
                           actual=mergeData.AggregateLoad[train_size+val_size:train_size+val_size+OUT_STEPS],
                           prices=prices)
modelPenalty

In [ ]:
# plot penalty by hour
x = np.arange(len(modelPenaltyHourly))
width = 0.3

plt.bar(x - 0.17, baselinePenaltyHourly, width, label='Baseline')
plt.bar(x + 0.17, modelPenaltyHourly, width, label='Model')
plt.ylabel(f'Penalty (GBP)')
plt.xlabel(f'Half-hour')
_ = plt.legend()
plt.title(f'Penalty by half-hour')
plt.savefig('Penalty by half-hour.png', format='png')

In [ ]:
modelBenefit = baselinePenalty - modelPenalty
modelBenefitPerCustomer = modelBenefit / 4987
print('\nBaseline penalty: ${:,.2f}'.format(baselinePenalty))
print('Model penalty: ${:,.2f}'.format(modelPenalty))
print('Net Model absolute benefit: ${:,.2f}'.format(modelBenefit))
print('Net Model benefit: {:,.2f}'.format((baselinePenalty - modelPenalty) / baselinePenalty * 100), '%')
print('Net Model benefit per customer: ${:,.2f}'.format(modelBenefitPerCustomer))